In [ ]:
!pip install --upgrade langchain 

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template("{topic}에 대한 간단한 농담을 말해줘.")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser # LCEL의 기본 파이프라인

chain.invoke({"topic": "아이스크림"})

/home/ubuntu/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'아이스크림이 눈치없이 녹는다고? 그렇다면... 그것은 참 아이스크림이다!'

In [3]:
!pip install langchain docarray tiktoken

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 KB 11.4 MB/s eta 0:00:00


In [5]:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["해리슨은 학교에서 일합니다.", "곰은 꿀을 좋아해"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """다음 지문에만 근거해서 질문에 답하세요:
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel( # 여러개의 Runnable을 병렬적으로 실행
    {"context": retriever, "question": RunnablePassthrough()} # RunnablePassthrough는 값을 입력받아 그대로 전달하는 객체
)
chain = setup_and_retrieval | prompt | model | output_parser

# retrieval | prompt | model | output. RAG의 기본 파이프라인

/home/ubuntu/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [6]:
chain.invoke("해리슨은 어디에서 일하나요?")

'답변: 학교에서 일합니다.'

In [7]:

chain.invoke("곰은 무엇을 좋아하나요?")

'답변: 곰은 꿀을 좋아합니다.'

In [10]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "{topic}에 대한 간단한 농담을 해줘."
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

왜 아이스크림이 항상 웃을까요? 

- 아이스크림은 항상 쌀쌀하니까요!

In [12]:
chain.batch(["아이스크림", "스파게티", "만두"])

# stream: 입력에 대해 청크를 스트리밍
# invoke: 입력에 대해 체인을 호출
# batch: 입력목록에 대해 체인을 배치로 호출

['어느 아이스크림이 가장 위험한가요? \n\n- 눈덩이 아이스크림! 와르르륵!',
 '왜 스파게티가 항상 신나게 춤을 출까요? \n\n파스타가 흥겨운 음악에 맞춰 노래하니까요!',
 '만두는 왜 좋은가요? \n\n만두는 만나고 싶은 사람이 많아서!']

In [13]:
await chain.ainvoke("ice cream")

# astream: 비동기로 입력에 대해 청크를 스트리밍
# ainvoke: 비동기로 입력에 대해 체인을 호출
# abatch: 비동기로 입력목록에 대해 체인을 배치로 호출

'아이스크림을 먹으면 뚜껑을 닫지 않아도 되요. 왜냐하면 곧 다 먹어버릴 테니까요!'

In [14]:
await chain.abatch(
    ["아이스크림", "스파게티", "만두"]
)

['어떤 아이스크림이 가장 빠를까요? 속도가 가장 빠른 아이스크림!',
 '왜 스파게티가 항상 행복한가요? \n\n파스타가 있기 때문이죠!',
 '만두가 자꾸만 뜨네요. 왜 그럴까요? 만두는 부유하기 때문이죠!']

In [15]:
chain.input_schema.schema()

{'title': 'RunnableParallel<topic>Input', 'type': 'object', 'properties': {}}

In [16]:
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}